# Código necesario para el procesamiento de datos

In [1]:
import requests
import pandas as pd
from pandas import ExcelWriter
from IPython.display import display, Markdown


pd.set_option('display.max_colwidth', None)  # Mostrar el contenido completo de las celdas
pd.set_option('display.max_rows', None)      # Mostrar todas las filas
pd.set_option('display.max_columns', None)   # Mostrar todas las columnas

# Código necesario para acceder a Sellekchem. Pendiente de separarlo en un módulo

In [2]:
%pip install -q google-colab-selenium[undetected]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 17.9 MB/s eta 0:00:00


In [3]:
import google_colab_selenium as gs
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# Configuración de Chrome
options = Options()
options.add_argument("--window-size=1920,1080")
options.add_argument("--disable-infobars")
options.add_argument("--disable-popup-blocking")
options.add_argument("--ignore-certificate-errors")
options.add_argument("--incognito")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")

def buscar_medicamento(termino):
    driver = gs.Chrome(options=options)

    try:
        driver.get("https://www.selleckchem.com/search.html")

        # Espera explícita para el campo de búsqueda
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "searchDTO.searchParam"))
        )
        search_box.clear()
        search_box.send_keys(termino + Keys.RETURN)

        # Espera explícita para los resultados
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "tr[name='productList']")))

        html = driver.page_source
        driver.quit()
        return html

    except Exception as e:
        print("❌ Error:", e)
        driver.quit()
        return None

def extraer_medicamentos(html):
    soup = BeautifulSoup(html, 'html.parser')
    medicamentos = []
    filas = soup.find_all('tr', attrs={'name': 'productList'})

    for fila in filas:
        td_catalogo = fila.find('td', class_='posRel')
        catalogo = td_catalogo.get_text(strip=True) if td_catalogo else "N/A"

        enlace = fila.find('a', class_='blue f15 bold')
        nombre_producto = enlace.get_text(strip=True) if enlace else "N/A"
        link = enlace.get("href") if enlace else "N/A"

        p_tags = fila.find_all('p')
        descripcion = p_tags[1].get_text(strip=True) if len(p_tags) > 1 else "No description"

        medicamentos.append({
            "Catalog No.": catalogo,
            "Product Name": nombre_producto,
            "Link": link,
            "Description": descripcion
        })

    return medicamentos

# ======================
# FUNCIÓN REUTILIZABLE
# ======================
def obtener_link_selleckchem(farmaco):
    """Obtiene el primer link relevante de Selleckchem para un fármaco
    Args:
        farmaco (str): Nombre del fármaco (ej: 'gemfibrozil')
    Returns:
        str: URL completa o None si hay error
    """
    try:
        html = buscar_medicamento(farmaco)
        if not html:
            return None

        productos = extraer_medicamentos(html)
        if productos:
            primer_link = f"www.selleckchem.com{productos[0]['Link']}"
            return primer_link
        return None

    except Exception as e:
        print(f"Error obteniendo {farmaco}: {str(e)}")
        return None

# =======
# PRUEBA
# =======
if __name__ == "__main__":
    test_farmaco = "Gemfibrozil"
    print(obtener_link_selleckchem(test_farmaco))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

www.selleckchem.com/products/Gemfibrozil(Lopid).html


# Definir la URL del endpoint de la API de Pharos

In [4]:
graphql_url = "https://pharos-api.ncats.io/graphql"

# Definir la consulta GraphQL

In [5]:
query = """
query ObtenerInfoVariante {
  target(q: { sym: "TTR" }) {
    nombre: name
    uniprot_ID: uniprot
    descripcion: description
    claseDiana: tdl
    secuencia: seq

    referenciaOMIM: mim {
      OMIM_ID: mimid
      nombre: term
    }

    ligandosConocidos: ligands {
      nombre: name
      ligando_ID: ligid
    }

    deLosCualesSonFarmacosAprobados: ligands(isdrug: true) {
      nombre: name
      ligando_ID: ligid
    }

    relacionProteinaProteina: ppis{
      target {
        nombre: name
        proteina_ID: sym
        secuencia: seq
        claseDiana: tdl
      }
      propiedadesRelacion: props {
        name
        value
      }
    }

    numeroDeViasPorFuente: pathwayCounts {
      fuente: name
      numVias: value
    }

    vias: pathways {
      viaPharos_ID: pwid
      nombre: name
      fuente: type
      fuente_ID: sourceID
      url
    }
  }
}
"""

# Hacer la petición a la API y extracción de datos

In [6]:
response = requests.post(graphql_url, json={"query": query})
data = response.json()

# Extraer los datos
if "data" in data and "target" in data["data"]:
    dataIsfetched = True
    target_data = data["data"]["target"]

    # Crear un DataFrame con información principal
    df_info = pd.DataFrame([{key: target_data[key] for key in ["nombre", "uniprot_ID", "descripcion", "claseDiana", "secuencia"]}])

    # Crear DataFrame de referencia OMIM
    df_omim = pd.DataFrame(target_data.get("referenciaOMIM", []))

    # Crear DataFrame de ligandos conocidos
    df_ligandos = pd.DataFrame(target_data.get("ligandosConocidos", []))

    df_ligandos['Selleckchem'] = df_ligandos['nombre'].apply(
    lambda x: obtener_link_selleckchem(x) or 'No encontrado'
    )

    # Crear DataFrame de fármacos aprobados
    df_farmacos = pd.DataFrame(target_data.get("deLosCualesSonFarmacosAprobados", []))

    df_farmacos['Selleckchem'] = df_farmacos['nombre'].apply(
    lambda x: obtener_link_selleckchem(x) or 'No encontrado'
    )

    # Crear DataFrame de relaciones proteína-proteína con priorización
    relaciones = []

    # Ordenar y filtrar las relaciones
    relaciones_raw = target_data.get("relacionProteinaProteina", [])

    # Definir prioridad de clases
    prioridad_clases = {
        "Tclin": 1,
        "Tchem": 2,
        "Tbio": 3,
        "Tdark": 4
    }

    # Función para ordenar
    def ordenar_por_clase(relacion):
        clase = relacion.get("target", {}).get("claseDiana", "")
        return prioridad_clases.get(clase, 5)  # 5 para clases desconocidas

    # Ordenar y tomar top 10
    relaciones_ordenadas = sorted(
        relaciones_raw,
        key=ordenar_por_clase
    )[:10]

    # Procesar solo las top 10
    for rel in relaciones_ordenadas:
        if "target" in rel and "propiedadesRelacion" in rel:
            # Filtrar propiedades por relevancia (p_wrong, p_ni, p_int, novelty)
            propiedades_filtradas = []
            for prop in rel["propiedadesRelacion"]:
                if prop["name"] in ["p_wrong", "p_ni"]:
                # if prop["name"] in ["p_wrong", "p_ni", "p_int", "novelty"] Para sacar los cuatro tipos
                    propiedades_filtradas.append(prop)

            # Ordenar propiedades por prioridad
            prioridad_propiedades = {
                "p_wrong": 1,
                "p_ni": 2
               # "p_int": 3,
               # "novelty": 4
            }
            propiedades_ordenadas = sorted(
                propiedades_filtradas,
                key=lambda x: prioridad_propiedades.get(x["name"], 5)
            )

            # Agregar solo las propiedades ordenadas
            for prop in propiedades_ordenadas:
                relaciones.append({
                    "Proteina": rel["target"]["nombre"],
                    "Proteina_ID": rel["target"]["proteina_ID"],
                    "Clase Diana": rel["target"]["claseDiana"],
                    "Propiedad": prop["name"],
                    "Valor": prop["value"]
                })

    df_relaciones = pd.DataFrame(relaciones)

    #Crear Dataframe de numeroDeViasPorFuente
    df_numero_vias_por_fuente = pd.DataFrame(target_data.get("numeroDeViasPorFuente", []))

    # Crear DataFrame de vías
    df_vias = pd.DataFrame(target_data.get("vias", []))

else:
    dataIsfetched = False

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

   # Mostrar los DataFrames

In [7]:
def guardar_en_excel():
    # Create an ExcelWriter object
    writer = pd.ExcelWriter('pharos_data.xlsx', engine='xlsxwriter')

    # Write each DataFrame to a different sheet
    df_info.to_excel(writer, sheet_name='Info', index=False)
    df_omim.to_excel(writer, sheet_name='OMIM', index=False)
    df_ligandos.to_excel(writer, sheet_name='Ligandos', index=False)
    df_farmacos.to_excel(writer, sheet_name='Farmacos', index=False)
    df_relaciones.to_excel(writer, sheet_name='Relaciones', index=False)
    df_vias.to_excel(writer, sheet_name='Vias', index=False)

    # Save the Excel file
    writer.save()
    print("DataFrames saved to pharos_data.xlsx")

In [8]:
def mostrar_dataframe(titulo, df):
      """ Muestra un DataFrame con título o un mensaje si está vacío """
      display(Markdown(f"## {titulo}"))
      if df.empty:
          display(Markdown("> ⚠️ No se han encontrado datos."))
      else:
          display(Markdown(df.to_markdown(index=False)))

In [9]:
if(dataIsfetched):
    mostrar_dataframe("1. Información Principal del Target", df_info)
    mostrar_dataframe("2. Referencias OMIM", df_omim)
    mostrar_dataframe("3. Ligandos Conocidos", df_ligandos)
    mostrar_dataframe("4. Fármacos Aprobados", df_farmacos)

    display(Markdown("## 5. Proteínas Interactuantes"))
    if df_relaciones.empty:
        display(Markdown("> ⚠️ No se han encontrado interacciones."))
    else:
        for name, df in df_relaciones.groupby("Proteina_ID"):
            mostrar_dataframe(f"### Interacción: {name}",df)
            display(Markdown("---"))

    mostrar_dataframe("6. Número de Vías por Fuente", df_numero_vias_por_fuente)
    mostrar_dataframe("7. Detalle de Vías Biológicas", df_vias)

else:
    print("No se encontraron datos para el target solicitado.")
    #guardar_en_excel

## 1. Información Principal del Target

| nombre        | uniprot_ID   | descripcion                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | claseDiana   | secuencia                                                                                                                                           |
|:--------------|:-------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------------|:----------------------------------------------------------------------------------------------------------------------------------------------------|
| Transthyretin | P02766       | This gene encodes one of the three prealbumins, which include alpha-1-antitrypsin, transthyretin and orosomucoid. The encoded protein, transthyretin, is a homo-tetrameric carrier protein, which transports thyroid hormones in the plasma and cerebrospinal fluid. It is also involved in the transport of retinol (vitamin A) in the plasma by associating with retinol-binding protein. The protein may also be involved in other intracellular processes including proteolysis, nerve regeneration, autophagy and glucose homeostasis. Mutations in this gene are associated with amyloid deposition, predominantly affecting peripheral nerves or the heart, while a small percentage of the gene mutations are non-amyloidogenic. The mutations are implicated in the etiology of several diseases, including amyloidotic polyneuropathy, euthyroid hyperthyroxinaemia, amyloidotic vitreous opacities, cardiomyopathy, oculoleptomeningeal amyloidosis, meningocerebrovascular amyloidosis and carpal tunnel syndrome. [provided by RefSeq, Aug 2017] | Tclin        | MASHRLLLLCLAGLVFVSEAGPTGTGESKCPLMVKVLDAVRGSPAINVAVHVFRKAADDTWEPFASGKTSESGELHGLTTEEEFVEGIYKVEIDTKSYWKALGISPFHEHAEVVFTANDSGPRRYTIAALLSPYSYSTTAVVTNPKE |

## 2. Referencias OMIM

|   OMIM_ID | nombre                                         |
|----------:|:-----------------------------------------------|
|    105210 | AMYLOIDOSIS, HEREDITARY, TRANSTHYRETIN-RELATED |
|    115430 | CARPAL TUNNEL SYNDROME                         |
|    145680 | HYPERTHYROXINEMIA, DYSTRANSTHYRETINEMIC        |
|    176300 | TRANSTHYRETIN                                  |

## 3. Ligandos Conocidos

...

---

## 6. Número de Vías por Fuente

| fuente              |   numVias |
|:--------------------|----------:|
| Reactome            |        21 |
| KEGG                |         1 |
| PathwayCommons: pid |         1 |

## 7. Detalle de Vías Biológicas

|   viaPharos_ID | nombre                                        | fuente              | fuente_ID     | url                                                                                                                           |
|---------------:|:----------------------------------------------|:--------------------|:--------------|:------------------------------------------------------------------------------------------------------------------------------|
|          31589 | Thyroid hormone synthesis                     | KEGG                | path:hsa04918 | http://www.kegg.jp/kegg-bin/show_pathway?hsa04918                                                                             |
|          56631 | FOXA2 and FOXA3 transcription factor networks | PathwayCommons: pid |               | http://apps.pathwaycommons.org/pathways?uri=http%3A%2F%2Fpathwaycommons.org%2Fpc11%2FPathway_eb2ae7c355dd9dfea83cd5c1b235ba29 |
|          62326 | Amyloid fiber formation                       | Reactome            | R-HSA-977225  | https://idg.reactome.org/PathwayBrowser/#/R-HSA-977225&FLG=TTR                                                                |
|          81853 | Disease                                       | Reactome            | R-HSA-1643685 | https://idg.reactome.org/PathwayBrowser/#/R-HSA-1643685&FLG=TTR                                                               |
|          82086 | Diseases associated with visual transduction  | Reactome            | R-HSA-2474795 | https://idg.reactome.org/PathwayBrowser/#/R-HSA-2474795&FLG=TTR                                                               |
|          82783 | Diseases of signal transduction               | Reactome            | R-HSA-5663202 | https://idg.reactome.org/PathwayBrowser/#/R-HSA-5663202&FLG=TTR                                                               |
|          86077 | Extracellular matrix organization             | Reactome            | R-HSA-1474244 | https://idg.reactome.org/PathwayBrowser/#/R-HSA-1474244&FLG=TTR                                                               |
|          89443 | G alpha (i) signalling events                 | Reactome            | R-HSA-418594  | https://idg.reactome.org/PathwayBrowser/#/R-HSA-418594&FLG=TTR                                                                |
|          92590 | GPCR downstream signalling                    | Reactome            | R-HSA-388396  | https://idg.reactome.org/PathwayBrowser/#/R-HSA-388396&FLG=TTR                                                                |
|         102827 | Immune System                                 | Reactome            | R-HSA-168256  | https://idg.reactome.org/PathwayBrowser/#/R-HSA-168256&FLG=TTR                                                                |